In [7]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout,Activation, Flatten
from keras.layers import MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.utils import np_utils

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Display our data shape/dimensions
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Now we one hot encode outputs
num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [8]:
#import keras
l2_reg = 0

model = Sequential()

# 1st Conv Layer
model.add(Conv2D(96, (11, 11), input_shape=x_train.shape[1:], 
                 padding='same',kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Conv Layer
model.add(Conv2D(256, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Conv Layer
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 4th Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# 5th Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


# 1st FC Layer
model.add(Flatten())
model.add(Dense(3072))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 2nd FC Layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 3rd FC Layer
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

print(model.summary())

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("CIFAR10_AlexNet_10-{epoch:02d}-{val_acc:.2f}.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1)

reduce_lr = ReduceLROnPlateau(monitor="val_loss",
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop, checkpoint]

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 32, 32, 96)        34944     
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 96)        384       
_________________________________________________________________
activation_17 (Activation)   (None, 32, 32, 96)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 256)       614656    
_________________________________________________________________
batch_normalization_10 (Batc (None, 16, 16, 256)       1024      
_________________________________________________________________
activation_18 (Activation)   (None, 16, 16, 256)       0         
__________

In [9]:
batch_size = 16
epochs = 10

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          callbacks=callbacks)

# Save model
model.save("TrainedModel/cifar10_AlexNet.h5")

# Evaluate model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 400s 8ms/step - loss: 1.5307 - acc: 0.4543 - val_loss: 1.3645 - val_acc: 0.5261

Epoch 00001: val_loss improved from inf to 1.36454, saving model to CIFAR10_AlexNet_10-01-0.53.h5
Epoch 2/10
50000/50000 [==============================] - 397s 8ms/step - loss: 1.1510 - acc: 0.6000 - val_loss: 1.0610 - val_acc: 0.6389

Epoch 00002: val_loss improved from 1.36454 to 1.06101, saving model to CIFAR10_AlexNet_10-02-0.64.h5
Epoch 3/10
50000/50000 [==============================] - 398s 8ms/step - loss: 0.9489 - acc: 0.6759 - val_loss: 0.9764 - val_acc: 0.6717

Epoch 00003: val_loss improved from 1.06101 to 0.97644, saving model to CIFAR10_AlexNet_10-03-0.67.h5
Epoch 4/10
50000/50000 [==============================] - 398s 8ms/step - loss: 0.8042 - acc: 0.7295 - val_loss: 0.7810 - val_acc: 0.7375

Epoch 00004: val_loss improved from 0.97644 to 0.78104, saving model to CIFAR10_AlexNet_10-0